# Policy Gradient in MetaDrive

The goals of this notebook are the following:
* Provide a brief theoretical background on the
    * discrete policy gradient
    * continuous policy gradient
* Introduce Metadrive, a RL environment to train self-driving automobiles.
* Implement a simple policy gradient algorithm (REINFORCE) for the problem of self driving cars. 

### Why Policy Gradient?
There are a plethora of RL algorithms out there. Why do we focus on the policy gradient?
1. **Simplicity:** The policy gradient is one of the simplest RL algorithms out there. It's easy to understand, and it's easy to implement.
2. **Direct Optimization:** The policy gradient directly optimizes the objective that we care about: the expected return. (Read on if you want to understand what this means.) Other algorithms, such as Q-learning, optimize a proxy objective, and then use that to optimize the expected return. This can lead to suboptimal performance.
3. **Both Continuous and Discrete Actions:** The policy gradient can be used to optimize policies with both continuous and discrete action spaces. Other algorithms, such as Q-learning, can only be used to optimize policies with discrete action spaces. 

## Prerequisites

In order to keep the length of these tutorials short, and their scope focused, we assume you have some prerequisite knowledge:

Here's a list of the topics you should be familiar with, alongside some sources that you can review for 
* Partial Derivatives and Gradients
    * https://www.khanacademy.org/math/multivariable-calculus/multivariable-derivatives/partial-derivative-and-gradient-articles/a/introduction-to-partial-derivatives
    * https://www.khanacademy.org/math/multivariable-calculus/multivariable-derivatives/partial-derivative-and-gradient-articles/a/the-gradient
* Probability Distributions
    * https://www.khanacademy.org/math/statistics-probability/random-variables-stats-library/random-variables-discrete/v/discrete-and-continuous-random-variables
    * https://www.khanacademy.org/math/statistics-probability/random-variables-stats-library/random-variables-continuous/v/random-variables
* Backprop
    * 3Blue1Brown's Backprop Video: https://www.youtube.com/watch?v=tIeHLnjs5U8
    * Andrei Karpathy's Micrograd Video: https://www.youtube.com/watch?v=VMj-3S1tku0 
* PyTorch
    * https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html
    * https://pytorch.org/tutorials/beginner/pytorch_with_examples.html

## RL Background

*Note: Part of these notes are adapted from the OpenAI Spinning Up [lecture notes](https://spinningup.openai.com/en/latest/spinningup/rl_intro3.html).*

Before we start our derivation of the discrete policy gradient, we need to define our problem as well as some key terminology that we'll be using throughout the article. 


### Problem Definition
Any problem that we solve using RL is defined as a repeated interaction between an agent and an environment. At each time step $t$, the agent receives an observation $o_t$ from the environment, and then selects an action $a_t$ to perform. After performing the action, the agent receives a reward $r_t$ from the environment, and the environment transitions to a new state $s_{t+1}$. The goal of the agent is to maximize the sum of rewards it receives over the course of the interaction.

This is illustrated in the figure below:

![rl_diagram](./agentenv_whitebg.png)

*(Image Source: Barto & Sutton, 2018)*

### Markov Decision Process
Mathematically, we can define the problem as a Markov Decision Process (MDP). An MDP is a tuple $(S, A, P, R, \gamma)$, where:
* $S$ is the set of possible states of the environment.
* $A$ is the set of possible actions of the agent.
* $P$ is the transition probability matrix, where $P_a(s, s') = \Pr(s_{t+1} = s' | s_t = s, a_t = a)$.
    * It represents the probability that state $s$ transitions to state $s'$ when the agent performs action $a$. 
* $R$ is the reward function, where $R_a(s, s') = \mathbb{E}[r_{t+1} | s_t = s, a_t = a, s_{t+1} = s']$.
    * It represents the expected reward that the agent receives when it transitions from state $s$ to state $s'$ after performing action $a$.
* $\gamma$ is the discount factor, where $\gamma \in [0, 1]$.
    * It represents the agent's preference for immediate rewards over future rewards.
    * The reward recieved $n$ timesteps into the future is multiplied by $\gamma^n$. Since $\gamma < 1$, this reduces their value at an exponential rate, the farther away they are from the present.

Let's look at a toy example to examine how we can break down problems into MDPs.

#### Gridworld as an MDP

**Problem Description**: The agent is placed in a $4 \times 4$ gridworld, and it can move up, down, left, or right. The agent receives a reward of $+1$ for reaching the goal state, and a reward of $-1$ for falling into the pit. The agent receives a reward of $-0.1$ for every other state. The agent's goal is to reach the goal state as quickly as possible.

Here's how we would define this problem as an MDP:
* $S = \{s_{1,1}, s_{1,2}, \dots, s_{4,4}\}$
* $A = \{up, down, left, right\}$
* $P_a(s, s') = \begin{cases} 1 & \text{if } s' = \text{next\_state}(s, a) \\ 0 & \text{otherwise} \end{cases}$
    * Where $\text{next\_state}(s, a)$ is defined as: 
    
        $
            \text{next\_state}(s_{x,y}, a) =
            \begin{cases}
                s_{x-1,y} & \text{if } a = \text{up} \text{ and } x > 1 \\
                s_{x+1,y} & \text{if } a = \text{down} \text{ and } x < 4 \\
                s_{x,y-1} & \text{if } a = \text{left} \text{ and } y > 1 \\
                s_{x,y+1} & \text{if } a = \text{right} \text{ and } y < 4 \\
                s_{x,y} & \text{otherwise}
            \end{cases}
        $

* $R_a(s, s') = \begin{cases} +1 & \text{if } s' = \text{goal} \\ -1 & \text{if } s' = \text{pit} \\ -0.1 & \text{otherwise} \end{cases}$
* $\gamma = 1$
    * The agent does not have a preference for immediate rewards over future rewards.

### Policies, Trajectories, and Returns

**Policy**: The agent is fully described by a policy. A policy is a function (usually denoted $\pi$) that takes in a state $s$ and returns a probability distribution over actions $a$.
* $\pi(a | s) = \Pr(a_t = a | s_t = s)$

**Trajectory**: A trajectory, usually denoted $\tau$ is a sequence of actions and states that the agent experiences over the course of an interaction with the environment.
* $\tau = (s_0, a_0, r_0, s_1, a_1, r_1, \dots, s_T, a_T, r_T)$

**Return**: A return, usually denoted $R(\tau)$ is the sum of rewards that the agent receives over the course of a trajectory. This is what we want to optimize.
* $R(\tau) = \sum_{t=0}^T r_t$

When we take into account the discount factor $\gamma$, we get the discounted return:
* $R_{\gamma}(\tau) = \sum_{t=0}^T \gamma^t r_t$

## Derivation of the Discrete Policy Gradient

**Note**: There is a difference between the discrete and continuous versions of the policy gradient. In this notebook, we focus only on the discrete case. We pick up with the continuous case in the second notebook.

With the above background in mind, we are now ready to directly tackle the derivation of the policy gradient.

For this derivation, we're assuming that $\gamma = 1$, so the return is undiscounted.

#### The Policy
Our policy is denoted $\pi_{\theta}$. The subscript $\theta$ indicates that our policy is parametrized by a set of neural network parameters, denoted $\theta$. We're trying to find a value for $\theta$ that makes the policy get a high reward. 

#### Objective Function
Let's start by writing down our **objective function**.
This function, typically denoted $J$, describes how good a particular policy $\pi_{\theta}$ is.
The objective function is what we want to maximize.

For the policy gradient, $J(\pi_{\theta})$ is just the expected value of the return over the distribution of trajectories that would be visited by the policy. Mathematically:
$$
    J(\pi_{\theta}) = \mathop{\mathbb{E}}_{\tau \sim \pi_{\theta}}\left[R(\tau)\right]
$$

Let's take a minute to expand this out so we understand it better.

Let:
$$
    \mathcal{T} = \text{the set of all possible trajectories that could be produced by following $\pi_{\theta}$}\
$$
Then:
$$
    J(\pi_{\theta}) = \sum_{\tau \in \mathcal{T}} R(\tau) \Pr(\tau|\pi_{\theta})
$$

Remember that both the policy and environment can be stochastic, so even if the starting condition is the same, there may be many possible trajectories.

Ok, now that we know what the objective function is, how do we optimize $\theta$ to maximize $J$? Since most problems have absurdly complex environments, there's no straightforward equation we can write and solve analytically. In the absence of an analytical solution, a good choice is gradient ascent. *(Note: since we want to maximize $J$, we're using gradient ascent. If we wanted to minimze loss, we would use gradient descent)*.

If you're confused about how gradient ascent (or descent) works, check out the linked resources in [Prequisites](#prerequisites).

When we're doing gradient ascent, our update rule for $\theta$ is:
$$
\theta_{k+1} = \theta_{k} + \alpha \nabla_{\theta} J(\pi_{\theta_k})
$$
Where $\theta_k$ current set of parameters, and $\theta_{k+1}$ is the next set of parameters.

The quantity $\nabla_{\theta} J(\pi_{\theta_k})$ is called the **policy gradient** and lends its name to this algorithm.

### Solving the Policy Gradient

Expand definition of J:
$$
\nabla_{\theta} J(\pi_{\theta}) = \nabla_{\theta} \mathop{\mathbb{E}}_{\tau \sim \pi_{\theta}}\left[R(\tau)\right]
$$
Expand definition of expectation:
$$
\nabla_{\theta} J(\pi_{\theta}) = \nabla_{\theta} \sum_{\tau \in \mathcal{T}} R(\tau) \Pr(\tau|\pi_{\theta})
$$
Move gradient inside sum:
$$
\nabla_{\theta} J(\pi_{\theta}) = \sum_{\tau \in \mathcal{T}} R(\tau) \nabla_{\theta} \Pr(\tau|\pi_{\theta})
$$

#### Detour: The Log Trick
We're going to take a quick detour to talk about the log trick. This is a useful trick that we'll use to simplify our derivation.

We start with the following basic identity:
$$
\frac{d}{dx} \ln f(x) = \frac{1}{f(x)} \left(\frac{d}{dx} f(x)\right)
$$
Rearrange the terms to isolate $\frac{d}{dx} f(x)$:
$$
\frac{d}{dx} f(x) = f(x) \frac{d}{dx} \ln f(x)
$$


#### Back to the Derivation
Applying this identity to our policy gradient, we get:
$$
\begin{align*}
\nabla_{\theta} J(\pi_{\theta}) &= \sum_{\tau \in \mathcal{T}} R(\tau) \left(\nabla_{\theta} \Pr(\tau|\pi_{\theta})\right)\\
&= \sum_{\tau \in \mathcal{T}} R(\tau) \left(\Pr(\tau|\pi_{\theta}) \nabla_{\theta} \ln \Pr(\tau|\pi_{\theta})\right)\\
&= \sum_{\tau \in \mathcal{T}} R(\tau) \nabla_{\theta} \ln \Pr(\tau|\pi_{\theta}) \Pr(\tau|\pi_{\theta}) 
\end{align*}
$$
Apply definition of expectation:
$$
\nabla_{\theta} J(\pi_{\theta}) = \mathop{\mathbb{E}}_{\tau \sim \pi_{\theta}}\left[R(\tau) \nabla_{\theta} \ln \Pr(\tau|\pi_{\theta})\right]
$$

#### Solving the Log Probability of a Trajectory

Let's focus on the term $\nabla_{\theta}\ln Pr(\tau|\pi_{\theta})$, from the previous equation. This is the gradient of the log probability of a trajectory. 

The probability of a given trajectory $\tau$ given $\pi_{\theta}$ is:
$$
\Pr(\tau|\pi_{\theta}) = \Pr(s_0) \prod_{t=0}^{T-1} \Pr(s_{t+1}|s_t, a_t) \pi_{\theta}(a_t|s_t)
$$
Where $T$ is the length of the trajectory.


Substituting this in, we can expand this out as follows:
$$
\begin{align*}
\nabla_{\theta} \ln \Pr(\tau|\pi_{\theta}) &= \nabla_{\theta} \ln \left(\Pr(s_0) \prod_{t=0}^{T-1} \Pr(s_{t+1}|s_t, a_t) \pi_{\theta}(a_t|s_t)\right)\\
&= \nabla_{\theta} \left(\ln \Pr(s_0) + \sum_{t=0}^{T-1} \ln \Pr(s_{t+1}|s_t, a_t) + \ln \pi_{\theta}(a_t|s_t)\right)\\
&= \nabla_{\theta} \sum_{t=0}^{T-1} \ln \pi_{\theta}(a_t|s_t)\\
&= \sum_{t=0}^{T-1} \nabla_{\theta} \ln \pi_{\theta}(a_t|s_t)
\end{align*}
$$